Read in the necessary imports:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy import linalg
from sklearn.preprocessing import StandardScaler

### Linear Algebra Python Intro

1.Let's see what the very important numpy and linalg packages can help us with. We can create matrices and multiply them together:

In [ ]:
A = np.matrix([[3,0],[8,-1]])
b = np.matrix([[1],[2]])
A*b

2.We can calculate the inverse, tranpose, and the determinant of a matrix:

In [ ]:
#inverse:
print(linalg.inv(A))

print()

#transpose:
print(A.T)

print()

#determinant:
print(linalg.det(A))

3.We can solve the system $Ax=b$:

In [ ]:
linalg.solve(A, b)

4.We can find the least squares solution and projection:

In [ ]:
#least squares
A=np.matrix([[1,1],[1,2],[1,3]])
b=np.matrix([[1],[2],[2]])

X=linalg.inv(A.T*A)*A.T*b
print(X) # solution
print()
print(A*X) # projection

5.We can also calculate the eigenvalues and eigenvectors of a matrix, which we'll get to more later:

In [ ]:
A=np.matrix([[3,0],[8,-1]])

#eigenvalues:
print(linalg.eigvals(A))

#eigenvalues and eigenvectors:
print(linalg.eig(A))

Note that the above output is read as $ \lambda_1 = -1, \lambda_2 = 3$ with eigenvectors $v_1 = [0,1],v_2=[0.447,0.894]$, which are the normalized versions of what you get from obtaining $v_1 = [0,1],v_2=[1,2]$ by hand.

I must admit that the output you get from typing ```eigenvectors(([[3,0],[8,-1]])``` into wolfram alpha is nicer.

### College Rankings

1.Read in the US News & World Report 2013 College Rankings:

In [ ]:
df = pd.read_csv('data/collegedata.csv')
df

2.Drop Grinnell since it does not contain SAT info:

In [ ]:
df = df[df['College'] != 'grinnell']

3.Save the Score column as series b and save the rest of the dataframe (except for the college and score columns) as A_dataframe:

In [ ]:
b = df['Score']
A_dataframe = df.drop(columns=['College', 'Score'])

4.Convert the dataframes to numpy matrices and then tranpose vector b so that the shapes are 25x14 and 25x1.

In [ ]:
A=np.matrix(A_dataframe)
b=np.matrix(b)
b = b.T
print(A.shape, b.shape)

5.Apply the least squares transformation:

In [ ]:
X=linalg.inv(A.T*A)*(A.T)*b

6.Print the weights in decending order:

In [ ]:
categories = A_dataframe.columns                   # column names

tuples = []                                        # create tuples containing the category weights and names
for i in range(len(categories)):
    tuples.append((X[i][0,0], categories[i]))
    
tuples.sort(reverse = True)                        # sort in decending order

for i in range(len(categories)):                   # print the output
    print(tuples[i])

7.What questions or observations do you have about the importance of these categories?

In [ ]:
#insert here

8.Print Colby's actual score and predicted score:

In [ ]:
#get colby's info
colby = df[df['College'] == 'colby']

#apply the least squares projection to colby's info
colby = colby.drop(columns = ['College', 'Score'])
colby = np.matrix(colby)
colby

projection = colby*X

#print the predicted and actual ranking
print('predicted ranking:', projection[0,0])
print('colby actual ranking:', df[df['College'] == 'colby']['Score'].values)

The average absolute error is:

In [ ]:
error = 0
for i in range(len(A)):
    projection = A[i][:]*X
    newerror = abs(projection[0,0] - b[i,0])
    error = error + newerror
    
print('Average Absolute Prediction Error:', error/len(A))

### A Discussion of Scaling

SAT Scores are on a much different scale than the other variables, so perhaps we should scale first? We have several options. We do NOT want to use the Standard scaler in this case because it causes the square matrix  $A^T A$ to have a determinant very close to zero, making it a nearly non-invertible matrix, which is bad.

In [ ]:
scaler = StandardScaler()

scaler.fit(A_dataframe)

A = np.matrix(scaler.transform(A_dataframe))

print(linalg.det(linalg.inv(A.T*A)))

We can try using the MinMax scaler instead, and this will no longer give us a zero determinat:

In [ ]:
scaler = MinMaxScaler()

scaler.fit(A_dataframe)

A = np.matrix(scaler.transform(A_dataframe))

print(linalg.det(linalg.inv(A.T*A)))

However, it still does not give us predictions as good as the non-scaled version. For example, the weightings are wackier and Colby's prediction is worse:

In [ ]:
X=linalg.inv(A.T*A)*(A.T)*b

categories = A_dataframe.columns                   # column names

tuples = []                                        # create tuples containing the category weights and names
for i in range(len(categories)):
    tuples.append((X[i][0,0], categories[i]))
    
tuples.sort(reverse = True)                        # sort in decending order

for i in range(len(categories)):                   # print the output
    print(tuples[i])
    
    
#get colby's info
colby = df[df['College'] == 'colby']

#apply the least squares projection to colby's info
colby = colby.drop(columns = ['College', 'Score'])



colby = scaler.transform(colby)   # don't scale because it gives worse results
colby = np.matrix(colby)
colby

projection = colby*X

#print the predicted and actual ranking
print('predicted ranking:', projection[0,0])
print('colby actual ranking:', df[df['College'] == 'colby']['Score'].values)

The average error is also worse:

In [ ]:
error = 0
for i in range(len(A)):
    projection = A[i][:]*X
    newerror = abs(projection[0,0] - b[i,0])
    error = error + newerror
    
print('Average Absolute Prediction Error:', error/len(A))

In summary, when using Ordinary Least Squares to solve for the closed form solution, feature scaling will NOT be necessary; in fact, it may make our predictions worse.

The exception is when you apply regularization (L1/L2 Ridge, Lasso, etc.) Then you should use feature scaling. However, in the above examples, we have not applied any regularization.

In the past, we used feature scaling for gradient descent in order to help the solution converge in a shorter period of time.

Speaking of gradient descent...

### Ordinary Least Squares vs. Gradient Descent

There are two ways to solve for an optimal regression solution. You can solve it via the analytical solution (OLS) or via an iterative algorithm such as gradient descent.

1.**Similarities** between the two methods:

- Both can be applied to linear regression models.
- Both are minimizing the sum of the squared residuals $$\sum_{i=1}^n(y^{(i)}_{\text{predicted}}-y^{(i)}_{\text{actual}})^2$$
- Both work for multivariate problems.

2.**Differences** between the two methods:

OLS directly calculates the solution by solving for the system of equations generated when setting all partial derivatives = 0. The whole process is analytical and generates a closed form solution.

In contrast, gradient descent starts from guessing the local min and proceeds by taking small steps along the direction of the steepest descent. It's numerical and iterative and when the step size is small enough, one expects the updated guess to approach the real local min (converge to the least squared solution). In addition, gradient descent is able to tackle a wider array of problems that are analytically unsolvable.

The OLS closed-form solution may (should) be preferred for “smaller” datasets – if computing (a “costly”) matrix inverse is not a concern. For very large datasets, or datasets where the inverse of $X^T X$ may not exist (the matrix is non-invertible or singular, e.g., in case of perfect multicollinearity), the GD or SGD approaches are to be preferred. Here's a good article going into more detail...

https://sebastianraschka.com/faq/docs/closed-form-vs-gd.html